In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

In [ ]:
!git clone https://github.com/dduygaucho/classifying-birds.git

Cloning into 'classifying-birds'...
remote: Enumerating objects: 51536, done.
remote: Counting objects: 100% (3877/3877), done.
remote: Compressing objects: 100% (3872/3872), done.
remote: Total 51536 (delta 4), reused 3877 (delta 4), pack-reused 47659
Receiving objects: 100% (51536/51536), 1.04 GiB | 13.89 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Checking out files: 100% (50593/50593), done.


In [ ]:
from keras.preprocessing.image import ImageDataGenerator


#Xception

In [ ]:
from keras.applications.xception import preprocess_input
# Creating generator for the training set
train_gen = ImageDataGenerator(
    preprocessing_function= preprocess_input,
    shear_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip= True
)

train_datagenerator = train_gen.flow_from_directory(
    '/content/classifying-birds/train',
    target_size= (224,224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 64, 
    shuffle = True
)


val_gen = ImageDataGenerator(
    preprocessing_function= preprocess_input,
    # shear_range = 0.1,
    # zoom_range = 0.1,
    # horizontal_flip= True
)

val_datagenerator = val_gen.flow_from_directory(
    '/content/classifying-birds/valid',
    target_size= (224,224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32, 
    shuffle = True
)



test_gen = ImageDataGenerator(
    preprocessing_function = preprocess_input
    
)

test_datagenerator = test_gen.flow_from_directory(
    '/content/classifying-birds/test',
    target_size= (224,224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32, 
    shuffle = False
)



Found 47332 images belonging to 325 classes.
Found 1625 images belonging to 325 classes.
Found 1625 images belonging to 325 classes.


##Build and train model

In [ ]:
base_model = keras.applications.Xception(
    include_top = False,
    weights = 'imagenet',
    input_shape = (224,224,3) # the input of the model is 224x224x3 (rgb)
)
# freeze all the params
base_model.trainable = False

83697664/83683744 [==============================] - 1s 0us/step


In [ ]:
model = keras.models.Sequential([
    base_model,
    keras.layers.Flatten(),
    keras.layers.Dense(1028, activation = 'selu', kernel_initializer= 'lecun_normal'),
    keras.layers.BatchNormalization(), 
    keras.layers.Dropout(0.75),
    keras.layers.Dense(512, activation = 'selu', kernel_initializer= 'lecun_normal'),
    keras.layers.BatchNormalization(), 
    keras.layers.Dropout(0.5),
    keras.layers.Dense(325, activation = 'softmax', kernel_initializer= 'glorot_normal')
])

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint('keras_xception_model.h5')
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 6, restore_best_weights= True)

model.compile(optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07), loss = 'categorical_crossentropy', metrics = ['accuracy']) # y_train already utils.to_categorical no need to use sparse (below)
history = model.fit(train_datagenerator, batch_size = 32, epochs = 22, validation_data = val_datagenerator, workers = 10, use_multiprocessing = True,
                    callbacks = [early_stopping_cb, checkpoint_cb])

Epoch 1/22
740/740 [==============================] - ETA: 0s - loss: 2.5805 - accuracy: 0.3963

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


740/740 [==============================] - 669s 882ms/step - loss: 2.5805 - accuracy: 0.3963 - val_loss: 0.7469 - val_accuracy: 0.7754
Epoch 2/22
153/740 [=====>........................] - ETA: 7:58 - loss: 1.5978 - accuracy: 0.5660

Process Keras_worker_ForkPoolWorker-50:
Traceback (most recent call last):
Process Keras_worker_ForkPoolWorker-42:
Process Keras_worker_ForkPoolWorker-49:
Process Keras_worker_ForkPoolWorker-44:
Process Keras_worker_ForkPoolWorker-43:
Process Keras_worker_ForkPoolWorker-46:
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process Keras_worker_ForkPoolWorker-47:
Process Keras_worker_ForkPoolWorker-48:
Process Keras_worker_ForkPoolWorker-45:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    sel

KeyboardInterrupt: ignored

In [ ]:
model.evaluate(test_datagenerator)

# VGG16

In [ ]:
from keras.applications.vgg16 import preprocess_input

In [ ]:
# Creating generator for the training set
train_gen = ImageDataGenerator(
    preprocessing_function= preprocess_input,
    shear_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip= True
)

train_datagenerator = train_gen.flow_from_directory(
    '/content/classifying-birds/train',
    target_size= (224,224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 64, 
    shuffle = True
)


val_gen = ImageDataGenerator(
    preprocessing_function= preprocess_input,
    # shear_range = 0.1,
    # zoom_range = 0.1,
    # horizontal_flip= True
)

val_datagenerator = val_gen.flow_from_directory(
    '/content/classifying-birds/valid',
    target_size= (224,224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32, 
    shuffle = True
)

Found 47332 images belonging to 325 classes.
Found 1625 images belonging to 325 classes.


## Build and train VGG model

In [ ]:
base_model = keras.applications.VGG16(
    include_top = False,
    weights = 'imagenet',
    input_shape = (224,224,3) # the input of the model is 224x224x3 (rgb)
)
# freeze all the params
base_model.trainable = False

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
model = keras.models.Sequential([
    base_model,
    keras.layers.Flatten(),
    keras.layers.Dense(1028, activation = 'selu', kernel_initializer= 'lecun_normal'),
    keras.layers.BatchNormalization(), 
    keras.layers.Dropout(0.75),
    keras.layers.Dense(512, activation = 'selu', kernel_initializer= 'lecun_normal'),
    keras.layers.BatchNormalization(), 
    keras.layers.Dropout(0.5),
    keras.layers.Dense(325, activation = 'softmax', kernel_initializer= 'glorot_normal')
])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 1028)              25791492  
                                                                 
 batch_normalization (BatchN  (None, 1028)             4112      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 1028)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               526848    
                                                        

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint('keras_model.h5')
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 6, restore_best_weights= True)
model.compile(optimizer = 'nadam', loss = 'categorical_crossentropy', metrics = ['accuracy']) # y_train already utils.to_categorical no need to use sparse (below)
history = model.fit(train_datagenerator, batch_size = 64, epochs = 30, validation_data = val_datagenerator, workers = 10, use_multiprocessing = True,
                    callbacks = [early_stopping_cb, checkpoint_cb])

Epoch 1/30
740/740 [==============================] - 641s 821ms/step - loss: 2.8047 - accuracy: 0.3888 - val_loss: 0.4682 - val_accuracy: 0.8732
Epoch 2/30
740/740 [==============================] - 584s 778ms/step - loss: 1.2184 - accuracy: 0.6733 - val_loss: 0.2762 - val_accuracy: 0.9243
Epoch 3/30
740/740 [==============================] - 581s 775ms/step - loss: 0.9628 - accuracy: 0.7379 - val_loss: 0.2659 - val_accuracy: 0.9274
Epoch 4/30
740/740 [==============================] - 582s 775ms/step - loss: 0.8383 - accuracy: 0.7680 - val_loss: 0.2079 - val_accuracy: 0.9366
Epoch 5/30
740/740 [==============================] - 580s 773ms/step - loss: 0.7598 - accuracy: 0.7841 - val_loss: 0.1832 - val_accuracy: 0.9452
Epoch 6/30
740/740 [==============================] - 602s 803ms/step - loss: 0.6987 - accuracy: 0.8020 - val_loss: 0.1699 - val_accuracy: 0.9458
Epoch 7/30
740/740 [==============================] - 587s 782ms/step - loss: 0.6450 - accuracy: 0.8140 - val_loss: 0.1579 -

In [ ]:
train_datagenerator[1][1][0] 

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

# Own CNN


In [ ]:
# Creating generator for the training set
train_gen = ImageDataGenerator(
    rescale = 1/255.,
    shear_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip= True
)

train_datagenerator = train_gen.flow_from_directory(
    '/content/classifying-birds/train',
    target_size= (224,224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32, 
    shuffle = True
)


val_gen = ImageDataGenerator(
    rescale = 1/255.
    # shear_range = 0.1,
    # zoom_range = 0.1,
    # horizontal_flip= True
)

val_datagenerator = val_gen.flow_from_directory(
    '/content/classifying-birds/valid',
    target_size= (224,224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32, 
    shuffle = False
)

test_gen = ImageDataGenerator(
    rescale = 1/255.
    # shear_range = 0.1,
    # zoom_range = 0.1,
    # horizontal_flip= True
)

test_datagenerator = test_gen.flow_from_directory(
    '/content/classifying-birds/test',
    target_size= (224,224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32, 
    shuffle = False
)



Found 47332 images belonging to 325 classes.
Found 1625 images belonging to 325 classes.
Found 1625 images belonging to 325 classes.


In [ ]:
len(train_datagenerator)

1480

In [ ]:
train_datagenerator[0] # a 2 element tuple, first ind = X_train, y_train
train_datagenerator[0][0].shape

(32, 224, 224, 3)

In [ ]:
# model = keras.models.Sequential()
# for i in range(15):
#   if i == 0:
#     model.add(keras.layers.Input(shape = (224,224,3)))
#   model.add(keras.layers.Conv2D(512, (2,2), activation = 'relu', padding = 'SAME'))
#   model.add(keras.layers.Conv2D(1024, (3,3), activation = 'relu', padding = 'SAME'))
#   model.add(keras.layers.MaxPool2D(padding = 'SAME'))
#   model.add(keras.layers.BatchNormalization())

# model.add(keras.layers.GlobalMaxPooling2D())
# model.add(keras.layers.Dense(450, activation = 'selu', kernel_initializer= 'lecun_normal'))
# model.add(keras.layers.Dense(325, activation = 'softmax', kernel_initializer= 'glorot_normal'))

from keras.layers import Conv2D, MaxPool2D, BatchNormalization, GlobalAvgPool2D, Dense, Flatten

def bird_model():
  my_input = keras.layers.Input(shape = (224,224,3))

  
  x = Conv2D(32, (3,3), activation= 'relu', padding = 'SAME')(my_input) # there are 32 filters, which are 3x3 size
  x = MaxPool2D( padding = 'SAME')(x)  # look for the best value
  x = BatchNormalization(momentum= 0.99)(x)

  initial = 64
  for i in range(7):
    
    x = Conv2D(initial, (3,3), activation= 'relu',  padding = 'SAME')(x)
    x = MaxPool2D(padding = 'SAME')(x)
    x = BatchNormalization()(x) # normalise the batch to prevent vanishing exploding
    initial *=2



  x = Flatten()(x) 
  x = BatchNormalization()(x)
  # x = GlobalAvgPool2D()(x) # Output the last element of the previous BN/ either use flatten or globalavgpool
  x = Dense(512, activation= 'selu', kernel_initializer = 'lecun_normal')(x)
  x = BatchNormalization()(x) 
  x = Dense(1024, activation = 'selu', kernel_initializer = 'lecun_normal')(x)
  x = Dense(325, activation= 'softmax')(x)

  model = keras.models.Model(inputs = my_input, outputs = x)
  # model = Model(... )
  return model


model = bird_model()




In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 32)     128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 64)       0     

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint('keras_mycnn_model.h5')
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 6, restore_best_weights= True)
model.compile(optimizer = 'nadam', loss=  'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(train_datagenerator, batch_size = 32, epochs = 25, validation_data = val_datagenerator, shuffle = True)

Epoch 1/25
1480/1480 [==============================] - 833s 551ms/step - loss: 4.0231 - accuracy: 0.1928 - val_loss: 3.1513 - val_accuracy: 0.3114
Epoch 2/25
1480/1480 [==============================] - 809s 546ms/step - loss: 2.3658 - accuracy: 0.4545 - val_loss: 3.1582 - val_accuracy: 0.4486
Epoch 3/25
1480/1480 [==============================] - 802s 541ms/step - loss: 1.9625 - accuracy: 0.5498 - val_loss: 1.5964 - val_accuracy: 0.6012
Epoch 4/25
1480/1480 [==============================] - 803s 542ms/step - loss: 1.6131 - accuracy: 0.6166 - val_loss: 1.0891 - val_accuracy: 0.7157
Epoch 5/25
1480/1480 [==============================] - 801s 541ms/step - loss: 1.1134 - accuracy: 0.7126 - val_loss: 244.7005 - val_accuracy: 0.7354
Epoch 6/25
1480/1480 [==============================] - 806s 544ms/step - loss: 0.8596 - accuracy: 0.7722 - val_loss: 54.5083 - val_accuracy: 0.7680
Epoch 7/25
1480/1480 [==============================] - 810s 547ms/step - loss: 0.7124 - accuracy: 0.8040 - v